# 3-D

In [ ]:
## imports
import os

import matplotlib.pyplot as plt
import numpy as np

from pycbem.bhte import (init_temp, delta_temp_analytic, delta_temp3)
from pycbem.constants import (eps_0, mu_0)
from pycbem.field import (efield, hfield)
from pycbem.utils.dataloader import (load_tissue_diel_properties,
                                     load_antenna_el_properties)
from pycbem.utils.integrate import elementwise_dblquad
from pycbem.utils.viz import fig_config

In [ ]:
## constants
# dry skin bhte parameters from IT'IS database
# https://itis.swiss/virtual-population/tissue-properties/database/
k = 0.37       # thermal conductivity [W/m/°C]
rho = 1109.    # dry skin density [kg/m^3]
C = 3391.      # specific heat of dry skin [Ws/kg/°C]
m_b = 1.76e-6  # blood perfusion [m^3/kg/s], equivalent to 106 mL/min/kg

# for blood
k_b = 0.52     # thermal conductivity [W/m/°C]
rho_b = 1000.  # kg/m^3
C_b = 3617.    # specific heat of blood [J/kg/°C]

# additional parameters
h_0 = 10.      # heat convection coefficient skin surface - air [W/m^2/°C]
T_a = 37.      # arterial temperature [°C]     
T_c = 37.      # body core temperature [°C]
T_f = 25.      # surrounding air temperature [°C]
Q_m = 33800.   # metabolic heat generation [W/m^3]

## 3-D bioheat transfer equation in time

Comparisson of the spectral method time domain solution and the analytic solution proposed in Foster et al. 2016.
This time, the model for the SMTD method is three-dimensional. Additionally, realistic antenna is considered -- half-wave dipole centrally powered by a voltage source set to 1V.

### Setup

In [ ]:
# frequency
f = 10e9

# conductivity, relative permitivity, tangent loss and penetration depth (Gabriel et al. 1996)
sigma, eps_r, tan_loss, pen_depth = load_tissue_diel_properties('skin_dry', f)

# `pen_depth` is the energy penetration depth into tissue, which is defined as the distance 
# beneath the surface at which the SAR has fallen to a factor of 1/e below that at the surface;
# one-half of the more commonly reported wave penetration depth
pen_depth = pen_depth / 2

# air (vacuum) resistance 
Z_air = np.sqrt(mu_0 / eps_0)

# dry skin resistance
Z_skin_dry = np.sqrt(mu_0 / (eps_r * eps_0))

# energy (Fresnel) transmission coefficient into the tissue
T_tr = 2 * Z_skin_dry / (Z_air + Z_skin_dry)

# antenna electric properties (Poljak 2005)
antenna_data = load_antenna_el_properties(f)
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j

# antenna position -- coordinates
xs = antenna_data.x.to_numpy()
ys = np.zeros_like(xs)
zs = np.zeros_like(xs)

# effective radiated skin area
target_area = (0.02, 0.02)  # 2 x 2 cm2
A = target_area[0] * target_area[1]
h = - 15 / 1000  # distance from the antenna
N = [11, 11, 11]
xt = np.linspace(-target_area[0]/2, target_area[0]/2, N[0]) + xs[-1] / 2
yt = np.linspace(-target_area[1]/2, target_area[1]/2, N[1])
zt = np.array([h])

### EM-field

In [ ]:
Ex = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
Ey = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
Ez = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
Hx = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
Hy = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
Hz = np.empty((xt.size, yt.size, zt.size), dtype=np.complex128)
for x_idx, _xt in enumerate(xt):
    for y_idx, _yt in enumerate(yt):
        for z_idx, _zt in enumerate(zt):
            ex, ey, ez = efield(_xt, _yt, _zt, xs, ys, zs, Is, f)
            hx, hy, hz = hfield(_xt, _yt, _zt, xs, ys, zs, Is, f)
            Ex[x_idx, y_idx, z_idx], Ey[x_idx, y_idx, z_idx], Ez[x_idx, y_idx, z_idx] = ex.item(), ey.item(), ez.item()
            Hx[x_idx, y_idx, z_idx], Hy[x_idx, y_idx, z_idx], Hz[x_idx, y_idx, z_idx] = hx.item(), hy.item(), hz.item()

In [ ]:
I0m = np.sqrt(np.power(Ey * Hz.conj() - Ez * Hy.conj(), 2)
              + np.power(Ex * Hz.conj(), 2)
              + np.power(Ex * Hy.conj(), 2))
I0 = np.abs(I0m)
S0 = 1 / 2 * np.real(I0m)

In [ ]:
IPD = 1 / A * elementwise_dblquad(I0, xt, yt, 31)
APD = 1 / A * elementwise_dblquad(S0, xt, yt, 31)

print(f'{IPD = }')
print(f'{APD = }')

### Simulation

In [ ]:
sim_time = 1080
tg = 1001
t = np.linspace(0, sim_time, tg)
SAR_sur = I0[:, :, 0] * T_tr / (rho * pen_depth)
deltaT = delta_temp3(t, N, target_area, pen_depth, k, rho, C, m_b , SAR_sur)

# save simulation into npy
#fname = (f'deltaT_source-HWdipole_'
#         f'grid-{N[0]}x{N[1]}x{N[2]}_'
#         f'surface-{int(target_area[0]*target_area[1]*1e4)}cm_'
#         f'timegrid-{tg}_'
#         f'simtime-{sim_time}s.npy')
#np.save(os.path.join('simulations', fname ), deltaT, allow_pickle=True, fix_imports=True)

### Visualization

Temperature change on the surface.

In [ ]:
fig_config(nrows=1, ncols=1, scaler=2.5)
fig = plt.figure()
ax = fig.add_subplot()
cs = ax.imshow(deltaT[-1, :, :, 0], origin='lower',
               extent=(xt.min(), xt.max(), yt.min(), yt.max()))
ax.plot(xs, ys, 'r-', linewidth=4, label='dipole antenna')
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel('$\\Delta T_{sur}$ [°C]')
ax.set_xlabel('$x$ [m]')
ax.set_ylabel('$y$ [m]')
ax.legend()
plt.tight_layout()
plt.show()

Temperature evolution on the surface and the distribution of temperature before (Deng2002) and after the exposure (depth-wise).

In [ ]:
z = np.linspace(0, pen_depth, N[2])
T0 = init_temp(z, k, rho, C, rho_b, C_b, m_b, h_0, T_a, T_c, T_f, Q_m)

fig_config(nrows=1, ncols=2, scaler=1.25)
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(t, np.ones_like(t) * T0[0] + delta_temp_analytic(t, pen_depth, k, rho, C, I0.max(), T_tr),
         'k--', label='$T_{sur}(t)$, $m_b = 0$, Foster et al.')
ax1.plot(t, np.ones_like(t) * T0[0] + np.max(deltaT[:, :, :, 0], axis=(1, 2)),
         'k-', label='$T_{sur}(t)$, 3-D PSTD')
ax1.set_xlabel('$t$ [s]')
ax1.set_ylabel('$T$ [°C]')
ax1.legend(loc='upper left')
ax1.grid()

ax2 = fig.add_subplot(1, 2, 2)
xargmax, yargmax = np.where(deltaT[-1, :, :, 0] == deltaT[-1, :, :, 0].max())
ax2.plot(z * 1e3, T0 + deltaT[-1, int(xargmax), int(yargmax), :], 'k-',
         label=f'$T(z, t={int(t[-1])})$, 3-D PSTD')
ax2.plot(z * 1e3, T0, 'k--', label='$T(z)$, Deng et al.')
ax2.set_xlabel('$z$ [mm]')
ax2.set_ylabel('$T$ [°C]')
ax2.legend(loc='center left')
ax2.grid()

plt.tight_layout()
plt.show()